In [1]:
%matplotlib inline
import torch
from torch.distributions import Normal
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.utils import save_image

In [2]:
directory = './'

In [3]:
torchvision.datasets.MNIST(directory, train=True, download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: E:/datasets/
    Split: Train

In [4]:
images, ground_truth = torch.load(directory + "MNIST/processed/training.pt")

In [5]:
print(images.shape)
print(ground_truth.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [6]:
print(torch.min(images[0]))
print(torch.max(images[0]))

tensor(0, dtype=torch.uint8)
tensor(255, dtype=torch.uint8)


In [7]:
image_index = 15

print(ground_truth[image_index])
# plt.imshow(images[image_index], cmap='gray')

tensor(7)


In [8]:
from torch.utils.data import Dataset

class MNISTDataset(Dataset):
    def __init__(self, path):
        self.images, self.ground_truth = torch.load(path)
    
    def __getitem__(self, idx):
        x = self.images[idx].float()
        x = torch.Tensor(x)
        x = torch.flatten(x)
        return x
    
    def __len__(self):
        return len(self.ground_truth)

In [9]:
train_dataset = MNISTDataset(directory + "MNIST/processed/training.pt")
test_dataset = MNISTDataset(directory + "MNIST/processed/test.pt")

print("Train dataset length:", len(train_dataset))
print("Test dataset length:", len(test_dataset))

Train dataset length: 60000
Test dataset length: 10000


In [10]:
train_dataset[15].shape

torch.Size([784])

In [14]:
import torch.nn as nn

class AutoEncoder(nn.Module):
    
    def __init__(self, input_shape, latent_dim=128):
        super().__init__()
        self.encoder_l1 = nn.Linear(in_features=input_shape, out_features=latent_dim)
        self.encoder_l2 = nn.Linear(in_features=latent_dim, out_features=latent_dim)
        
        self.decoder_l1 = nn.Linear(in_features=latent_dim, out_features=latent_dim)
        self.decoder_l2 = nn.Linear(in_features=latent_dim, out_features=input_shape)
        
    def forward(self, x):
        latent = self.run_encoder(x)
        x_hat = self.run_decoder(latent)
        return x_hat
    
    def run_encoder(self, x):
        output = F.relu(self.encoder_l1(x))
        latent = F.relu(self.encoder_l2(output))
        return latent
    
    def run_decoder(self, latent):
        output = F.relu(self.decoder_l1(latent))
        x_hat = F.relu(self.decoder_l2(output))
        return x_hat

In [15]:
model = AutoEncoder(input_shape=train_dataset[0].shape[0])
model.cuda()

AutoEncoder(
  (encoder_l1): Linear(in_features=784, out_features=128, bias=True)
  (encoder_l2): Linear(in_features=128, out_features=128, bias=True)
  (decoder_l1): Linear(in_features=128, out_features=128, bias=True)
  (decoder_l2): Linear(in_features=128, out_features=784, bias=True)
)

In [16]:
model.train()

AutoEncoder(
  (encoder_l1): Linear(in_features=784, out_features=128, bias=True)
  (encoder_l2): Linear(in_features=128, out_features=128, bias=True)
  (decoder_l1): Linear(in_features=128, out_features=128, bias=True)
  (decoder_l2): Linear(in_features=128, out_features=784, bias=True)
)

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True)
mse = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 100

for epoch in range(n_epochs):
    loss = 0
    
    for batch_features in train_loader:
        batch_features = batch_features.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(batch_features)
        
        train_loss = mse(outputs, batch_features)
        
        train_loss.backward()
        
        optimizer.step()
        
        loss += train_loss.item()
    
    loss = loss / len(train_loader)
    
    if epoch % 5 == 0:
        print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, n_epochs, loss))

print("epoch: {}/{}, loss = {:.6f}".format(epoch + 1, n_epochs, loss))

epoch : 1/100, loss = 2066.458537
epoch : 6/100, loss = 903.045576
epoch : 11/100, loss = 823.641144
epoch : 16/100, loss = 788.662906
epoch : 21/100, loss = 766.621867
epoch : 26/100, loss = 752.876644
epoch : 31/100, loss = 742.759293
epoch : 36/100, loss = 733.817733
epoch : 41/100, loss = 729.049442
epoch : 46/100, loss = 725.443281
epoch : 51/100, loss = 722.037865
epoch : 56/100, loss = 717.985390
epoch : 61/100, loss = 715.416301
epoch : 66/100, loss = 713.759525
epoch : 71/100, loss = 711.816621
epoch : 76/100, loss = 710.086370
epoch : 81/100, loss = 708.871276
epoch : 86/100, loss = 706.901524
epoch : 91/100, loss = 706.631080
epoch : 96/100, loss = 706.380368
epoch: 100/100, loss = 705.344930


In [18]:
model.eval()

image_idx = 100

image = train_dataset[image_idx].cuda()
x_hat = model(image)
x_hat = x_hat.detach().cpu().numpy().reshape((28, 28))
x_hat = np.around(x_hat)

In [19]:
# Get Latent Representation

@torch.no_grad()
def get_latent_representation(model, dataloader):
    latent_representation = []
    for batch in dataloader:
        batch = batch.cuda()
        latent = model.run_encoder(batch)
        latent = latent.cpu().numpy()
        latent_representation.append(latent)
    latent_representation = np.concatenate(latent_representation)
    return latent_representation

latent = get_latent_representation(model, train_loader)
latent.shape

(60000, 128)

In [20]:
# Variational Auto Encoder tingz

kernel_size = 4
init_channels = 8
image_channels = 1
latent_dim = 16

In [21]:
class ConvVAE(nn.Module):
    def __init__(self,
                 image_channels=1,
                 kernel_size=4,
                 latent_dim=16,
                 init_channels=8):
        super(ConvVAE, self).__init__()
        self.enc1 = nn.Conv2d(
            in_channels=image_channels, out_channels=init_channels, 
            kernel_size=kernel_size, stride=2, padding=1
        )
        self.enc2 = nn.Conv2d(
            in_channels=init_channels, out_channels=init_channels*2, 
            kernel_size=kernel_size, stride=2, padding=1
        )
        self.enc3 = nn.Conv2d(
            in_channels=init_channels*2, out_channels=init_channels*4, 
            kernel_size=kernel_size, stride=2, padding=1
        )
        self.enc4 = nn.Conv2d(
            in_channels=init_channels*4, out_channels=64, 
            kernel_size=kernel_size, stride=2, padding=0
        )
        
        self.fc1 = nn.Linear(64, 128)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_log_var = nn.Linear(128, latent_dim)
        self.fc2 = nn.Linear(latent_dim, 64)
        
        self.dec1 = nn.ConvTranspose2d(
            in_channels=64, out_channels=init_channels*8,
            kernel_size=kernel_size, stride=1, padding=0
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_channels*8, out_channels=init_channels*4,
            kernel_size=kernel_size, stride=2, padding=1
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_channels*4, out_channels=init_channels*2,
            kernel_size=kernel_size, stride=2, padding=1
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_channels*2, out_channels=image_channels,
            kernel_size=kernel_size, stride=2, padding=1
        )
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        sample = mu + (eps * std)
        return sample
    
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        batch, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        hidden = self.fc1(x)
        
        mu = self.fc_mu(hidden)
        log_var = self.fc_log_var(hidden)
        
        z = self.reparameterize(mu, log_var)
        z = self.fc2(z)
        z = z.view(-1, 64, 1, 1)
        
        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        reconstruction = torch.sigmoid(self.dec4(x))
        return reconstruction, mu, log_var

In [25]:
def final_loss(bce_loss, mu, logvar):
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def train(model, dataloader, dataset_size, device, optimizer, criterion):
    model.train()
    running_loss = 0.0
    counter = 0
    for i, data in tqdm(enumerate(dataloader),
                        total=int(dataset_size/dataloader.batch_size)):
        counter += 1
        data = data[0]
        data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    train_loss = running_loss / counter
    return train_loss

def validate(model, dataloader, dataset, device, criterion):
    model.eval()
    running_loss = 0.0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), 
                            total=int(len(dataset)/dataloader.batch_size)):
            counter += 1
            data = data[0]
            data = data.to(device)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
    
    val_loss = running_loss / counter
    return val_loss

In [26]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model = ConvVAE().to(device)

lr = 0.001
epochs = 50
batch_size = 64

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(
    root=directory, train=True, download=False, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True
)

testset = torchvision.datasets.MNIST(
    root=directory, train=False, download=False, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=True
)

In [27]:
optimizer = optim.Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss(reduction='sum')

for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    
    train_epoch_loss = train(
        model, trainloader, len(trainset), device, optimizer, criterion
    )
    valid_epoch_loss = validate(
        model, testloader, testset, device, criterion
    )
    
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")

Epoch 1 of 50


938it [00:16, 57.10it/s]                                                                               
157it [00:01, 80.88it/s]                                                                               


Train Loss: 15595.7613
Val Loss: 11625.4322
Epoch 2 of 50


938it [00:16, 56.40it/s]                                                                               
157it [00:01, 85.48it/s]                                                                               


Train Loss: 11197.3220
Val Loss: 10927.7700
Epoch 3 of 50


...

Train Loss: 9557.1179
Val Loss: 9557.8264
Epoch 49 of 50


938it [00:16, 55.94it/s]                                                                               
157it [00:02, 76.00it/s]                                                                               


Train Loss: 9552.2973
Val Loss: 9560.2611
Epoch 50 of 50


938it [00:16, 56.34it/s]                                                                               
157it [00:01, 92.34it/s]                                                                               

Train Loss: 9548.3431
Val Loss: 9580.2658
